**Introduction**

TensorFlow Serving models are consumed by clients. Client code needs to make a connection to the server and communicate using protocol buffers. Google's gRPC remote procedure call library provides the ability to connect and make calls to the server. After establishing a connection, you make a call by preparing the input for model prediction and serializing it in the request. The serialized prediction output is returned by the server.

In this Lab Step, you will use Python client code to request predictions from the model being served by TensorFlow Serving in the previous Lab Step.

In [1]:
from grpc.beta import implementations
import numpy as np
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2

hostport = 'localhost:9000'


def do_prediction(hostport):

  # Create connection
  host, port = hostport.split(':')
  channel = implementations.insecure_channel(host, int(port))
  stub = prediction_service_pb2.beta_create_PredictionService_stub(channel)

  # Initialize a request
  request = predict_pb2.PredictRequest()
  request.model_spec.name = 'nn'
  request.model_spec.signature_name = 'prediction'

  # Use evenly-spaced points for test data
  tests = temp_data = np.array([range(-1, 6, 1)]).transpose().astype(
    np.float32)

  # Set the tests as the input for prediction
  request.inputs['input'].CopyFrom(
    tf.contrib.util.make_tensor_proto(tests, shape=tests.shape))

  # Get prediction from server
  result = stub.Predict(request, 5.0) # 5 second timeout

  # Compare to noise-free actual values
  actual = np.sum(0.5 * temp_data + 2.5, 1)

  return result, actual


prediction, actual = do_prediction(hostport)
print('Prediction is: ', prediction)
print('Noise-free value is: ', actual)

ModuleNotFoundError: No module named 'tensorflow_serving'

The float_vals in the prediction output protocol buffer are the predicted values for each of the inputs you requested. Your values will differ from the image above, but you should see that they are close to the noise-free values.

 

**Summary
In this Lab Step, you created a TensorFlow Serving client that makes predictions using the model served in the previous Lab Step.

You have now seen a comprehensive view of working with TensorFlow using the Amazon Deep Learning AMI. You developed a model in TensorFlow, analyzed the learning process in TensorBoard, served the trained model with TensorFlow Serving, and consumed the model using Python client code. You would follow the same steps for adding your own model into production.